*This notebook is part of  course materials for CS 345: Machine Learning Foundations and Practice at Colorado State University.
Original versions were created by Asa Ben-Hur.
The content is availabe [on GitHub](https://github.com/asabenhur/CS345).*

*The text is released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/), and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<a href="https://colab.research.google.com/github/asabenhur/CS345/blob/master/fall24/notebooks/module03_04_multivariate_linear_regression.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Multivariate Linear Regression

*Adapted from Chapter 3 of [An Introduction to Statistical Learning](https://www.statlearning.com/)*

In [14]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)
import matplotlib.pyplot as plt

### Linear regression with multiple features

Univariate linear regression can easily be extended to include multiple features, leading to **multivariate linear regression**.  Instead of using a single variable to make a prediction, we use a vector of variables:

$$
\hat{y} = w_1x_1 + \ldots + w_dx_d + b = \mathbf{w}^\top \mathbf{x} + b
$$

As usual the variables $x_i$ represents the features, and each feature is associated a parameter $w_i$.  In our advertising data for example, $\mathbf{x} = (x_1,x_2, x_3)^\top$, and $\mathbf{w} = (w_1,w_2, w_3)^\top$ for TV, radio and newspaper advertising, respectively.

As in the univariate case, the parameters are chosen to minimize the sum-squared error:
$$
J( \mathbf{w},b ) = \sum_{i=1}^N (y_i - \hat{y}_i)^2,
$$
where 
$$
\hat{y}_i = \mathbf{w}^\top \mathbf{x}_i + b.
$$

Let's demonstrate on the advertising data:

In [15]:
# read data into a pandas DataFrame
data = pd.read_csv('https://www.statlearning.com/s/Advertising.csv', index_col=0)
data.head()

TV  radio  newspaper  sales
1  230.1   37.8       69.2   22.1
2   44.5   39.3       45.1   10.4
3   17.2   45.9       69.3    9.3
4  151.5   41.3       58.5   18.5
5  180.8   10.8       58.4   12.9

In [16]:
from sklearn.linear_model import LinearRegression

# create X and y
X = data[['TV', 'radio', 'newspaper']].values
y = data['sales'].values

# instantiate and fit
linreg = LinearRegression()
linreg.fit(X, y)

# print the coefficients
print ('bias: ', linreg.intercept_)
print ('weight vector: ', linreg.coef_)

bias:  2.9388893694593996
weight vector:  [ 0.0458  0.1885 -0.001 ]


For a given amount of Radio and Newspaper spending, an increase of $1000 in **TV** spending is associated with an **increase in sales of 45.8 widgets**.

For a given amount of TV and Newspaper spending, an increase of $1000 in **Radio** spending is associated with an **increase in sales of 188.5 widgets**.

For a given amount of TV and Radio spending, an increase of $1000 in **Newspaper** spending is associated with an **decrease in sales of 1.0 widgets**. How could that be?

## Evaluation metrics for regression problems

We just introduced the concept of fitting a multivariate linear model, let us now take a moment to ask what it might mean to judge the qaulity of a model after fitting.

Evaluation metrics for classification problems, such as **accuracy**, are not applicable for regression problems. We need evaluation metrics designed for comparing **continuous values**.

Let's create some example numeric predictions, and calculate three common evaluation metrics for regression problems:

In [17]:
# define true and predicted response values
y = [100, 50, 30, 20]
y_pred = [90, 50, 50, 30]

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:

$$\frac 1N\sum_{i=1}^N |y_i-\hat{y}_i|$$

It is implemented in scikit-learn:

In [18]:
from sklearn import metrics

metrics.mean_absolute_error(y, y_pred)

10.0

But it's just as easy to implement it yourself:

In [20]:
def mean_absolute_error(y, y_pred) :
    return np.mean(np.absolute(np.asarray(y) - np.asarray(y_pred)))

mean_absolute_error(y, y_pred)

10.0

**Mean Squared Error** (MSE) is the mean of the squared errors:

$$\frac 1N\sum_{i=1}^N(y_i-\hat{y}_i)^2$$

In [21]:
metrics.mean_squared_error(y, y_pred)

150.0

**Root Mean Squared Error** (RMSE) is the square root of the MSE:

$$\sqrt{\frac 1N\sum_{i=1}^N(y_i-\hat{y}_i)^2}$$

In [22]:
np.sqrt(metrics.mean_squared_error(y, y_pred))

12.24744871391589

Very easy to implement as well:

In [9]:
def root_mean_square_error(y, y_pred) :
    return np.sqrt(np.mean( (np.asarray(y) - np.asarray(y_pred))**2 ))

root_mean_square_error(y, y_pred)

12.24744871391589

Comparing these metrics:

- **MAE** is the easiest to understand, because it's the average error.
- **MSE** "punishes" larger errors
- **RMSE** easier to understand than MSE because RMSE is in the "y" units.

All of these are measures of **error** or loss, where lower is better.

Here's an example, to demonstrate how MSE/RMSE punish larger errors:

In [24]:
# same true values as above
y_true = [100, 50, 30, 20]

# new set of predicted values
y_pred = [60, 50, 30, 20]

# the previous values were:  y_pred = [90, 50, 50, 30]

# MAE is the same as before
print(f"MAE is  : {mean_absolute_error(y, y_pred):6.3f}")

# RMSE is larger than before
print(f"RMSE is : {root_mean_square_error(y, y_pred):6.3f}")

MAE is  : 10.000
RMSE is : 20.000


### Evaluating multivariate linear regression on the advertising data


In [26]:
from sklearn.model_selection import train_test_split

X = data[['TV', 'radio', 'newspaper']].values
y = data['sales'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=5)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred = linreg.predict(X_test)

# evaluation on the test set
mae_test = mean_absolute_error(y_test, y_pred)
print(f"MAE on the test set : {mae_test:6.3f}")

rmse_test = root_mean_square_error(y_test, y_pred)
print(f"RMSE on the test set : {rmse_test:6.3f}")

# evaluation on the training set
y_train_pred = linreg.predict(X_train)

mae_train = mean_absolute_error(y_train, y_train_pred)
print(f"MAE on the training set : {mae_train:6.3f}")

rmse_train = root_mean_square_error(y_train, y_train_pred)
print(f"RMSE on the training set : {rmse_train:6.3f}")

MAE on the test set :  1.216
RMSE on the test set :  1.583
MAE on the training set :  1.259
RMSE on the training set :  1.710


### Exercise:  is my regression method better than simply guessing a value?

Recall that in classification problems, a good classifier should do better than a classifier that simply predicts the class of the majority.
Is there a corresponding simple regression method?  Yes - in this case, simply predict the mean value of $y$.  Compare the value of the RMSE we obtained above with the value of the RMSE for this very simple regression method.  Are we doing better?

In [27]:
# compute the mean value of y and compute the RMSE of a regression 
# method that predicts that as the label
y_mean = np.mean(y_train)
y_mean

13.948571428571428

In [28]:
mean_absolute_error(y_mean, y_test)

3.8449999999999998

### Exercise:  Selecting good features

Split the data into train/test sets and use RMSE to decide whether the newspaper feature should be used for our model.  You will need to train/test two versions of the model - with and without that feature.

In [13]:
# for convenience, we'll recreate the data:

X = data[['TV', 'radio', 'newspaper']].values
y = data['sales'].values


In [29]:
from sklearn.model_selection import train_test_split

X = data[['TV', 'radio']].values
y = data['sales'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=5)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred = linreg.predict(X_test)

# evaluation on the test set
mae_test = mean_absolute_error(y_test, y_pred)
print(f"MAE on the test set : {mae_test:6.3f}")

rmse_test = root_mean_square_error(y_test, y_pred)
print(f"RMSE on the test set : {rmse_test:6.3f}")

# evaluation on the training set
y_train_pred = linreg.predict(X_train)

mae_train = mean_absolute_error(y_train, y_train_pred)
print(f"MAE on the training set : {mae_train:6.3f}")

rmse_train = root_mean_square_error(y_train, y_train_pred)
print(f"RMSE on the training set : {rmse_train:6.3f}")

MAE on the test set :  1.215
RMSE on the test set :  1.582
MAE on the training set :  1.261
RMSE on the training set :  1.711


### Advantages/disadvantages of linear regression

Advantages of linear regression:

- Simple to explain
- Highly interpretable
- Model training and prediction are fast
- No tuning is required 
- Can perform well with a small number of observations

Disadvantages of linear regression:

- Presumes a linear relationship between the features and the labels
- Performance is (generally) not competitive with the best regression methods
- Can be sensitive to irrelevant features and outliers

Linear regression is a **parametric method**, meaning that success depends on the data satisfying our assumption that the data fall on a line/hyperplane.